In [2]:
! pip install langchain-openai langchain-community langchainhub gpt4all langchain-chroma chromadb langchain pypdf

  Using cached langchain_openai-0.1.15-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_community-0.2.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchainhub-0.1.20-py3-none-any.whl.metadata (659 bytes)
  Using cached gpt4all-2.7.0-py3-none-manylinux1_x86_64.whl.metadata (4.7 kB)
  Using cached langchain_chroma-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached chromadb-0.5.4-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain-0.2.7-py3-none-any.whl.metadata (6.9 kB)
  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached langchain_core-0.2.13-py3-none-any.whl.metadata (6.0 kB)
  Using cached openai-1.35.13-py3-none-any.whl.metadata (21 kB)
  Using cached tiktoken-0.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.31-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_

In [ ]:
# Clone books
# ! git clone https://github.com/aridiosilva/AI_Books.git

In [3]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

# Load entire directory
# loader = DirectoryLoader("AI_Books/", glob="*.pdf", loader_cls=PyPDFLoader, use_multithreading=True)

# Load single file
loader = PyPDFLoader("https://arxiv.org/pdf/2406.12566", extract_images=False)
pages = loader.load()

In [4]:
# Get Text from all pages
txt = ' '.join([d.page_content for d in pages])

In [5]:
print("Pages: ",len(pages))
print("Text lenght: ", len(txt))

Pages:  16
Text lenght:  63473


In [6]:
# Split Text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 250,
    add_start_index=True
)

In [7]:
chunks = text_splitter.split_documents(pages)

In [8]:
# Count chunks
len(chunks)

87

## Vectorstore

In [9]:
! pip install chromadb

In [ ]:
!rm -rf docs1/

In [27]:
# Embedding with Azure OpenAI Ada
from langchain_chroma import Chroma
from langchain_community.embeddings import AzureOpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
import os

# Set environment variables for Azure OpenAI
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-35-turbo"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

# Define your embedding function using Azure OpenAI
embedding_function = AzureOpenAIEmbeddings(deployment="text-embedding-ada-002")

# Set the persist directory
persist_directory = './docs/'

# Create the Chroma vector database from documents and save it
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory=persist_directory
)

print(f"Saved chunks to {persist_directory}")

Saved chunks to ./docs/


## Prepare DB

In [14]:
db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_function
)

### Small test

In [15]:
print(db._collection.count())

87


In [16]:
query_text = "datasets used"

In [17]:
# k is number of results we want to return
results = db.similarity_search_with_score(query_text, k=3)

if len(results) == 0 or results[0][1] < 0.2:
    print("Unable to find matching results.")
else:
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {context_text}\nSources: {sources}"
    print(formatted_response)

Response: 4 Experiment
4.1 Datasets and Metrics
Datasets . We conduct our experiments on
two publicly available datasets that focus on
multi-document summarization and long-form
query-answer (QA) respectively, i.e.,WikiPas-
sageQA (Hayashi et al., 2021) and Wiki-
Asp (Hayashi et al., 2021). WikiPassageQA offers
human-annotated quality-evaluated questions and
long-form answers. We chose this dataset because
its answers are generally comprehensive, related
to various aspects of questions, and the answer
length is fairly long. WikiAsp dataset is devised
for generating aspect-based summaries of entities
from 20 domains. We follow (Jiang et al., 2023) to
convert it into open-domain QA settings. To sup-
port our experiments, we first operate some data
pre-processing to ensure that each piece of data con-
tains the question, ground truth answer, sub-aspects
of the question, and their sub-answers. The process
details and statistical information of datasets are
demonstrated in Appendix A.

---


## RAG

In [28]:
from langchain.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import os

model = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

conversation = ConversationChain(
    llm=model,
    verbose=False,
    memory=ConversationBufferMemory()
)

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
DO NOT give irelevant information that is not in the context.
---
Answer the question based on the above context: {question}
"""

/workspaces/promptSummer/.conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [29]:
query_text = "What are the datasets that have been used for the training?"

In [30]:
results = db.similarity_search_with_score(query_text, k=3)

In [31]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)

response_text = conversation.predict(input=prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: The datasets used for the experiments are WikiPassageQA and WikiAsp, which focus on multi-document summarization and long-form query-answer respectively. WikiPassageQA offers human-annotated quality-evaluated questions and long-form answers, while WikiAsp is devised for generating aspect-based summaries of entities from 20 domains. Both datasets are publicly available and were pre-processed to ensure that each piece of data contains the question, ground truth answer, sub-aspects of the question, and their sub-answers.
Sources: ['https://arxiv.org/pdf/2406.12566', 'https://arxiv.org/pdf/2406.12566', 'https://arxiv.org/pdf/2406.12566']
